# Getting smiles codes 🙂

1. Obtener lista de antivirales con scrapeo de BD
2. Crear una columna de SMILES de esos antivirales
3. Pasar la columna de SMILES a la función de maic con selenium para obtener la afinidad
4. Crear una columna con esos valores

In [1]:
import pubchempy as pcp
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup

In [2]:
def getSmilesByCID(cid):
    c = pcp.Compound.from_cid(cid)
    return c.canonical_smiles

def getSmilesByName(name):
    for compound in pcp.get_compounds(name, 'name'):
        return compound.canonical_smiles

In [4]:
page_number = 1

nombres = []
codigos = []

while page_number < 25:
    
    page_number += 1
    
    url = "https://www.drugbank.ca/unearth/q?approved=1&c=_score&ca=0&d=down&eu=0&page={}&query=antiviral&searcher=drugs&us=0".format(page_number)
    data = requests.get(url).text
    soup = BeautifulSoup(data, 'html.parser')
    hits = soup.find_all("h2",{"class":"hit-link"})
    
    for i in hits:
        #Crear la lista con sólo los nombres
        match = re.findall(r'DB\d{5}..(.*)</a></h2>', str(i))
        nombres.append(match)
        #Crear la lista con sólo los códigos
        db_code = re.findall(r'DB\d{5}', str(i))
        codigos.append(db_code)


In [5]:
print(len(nombres))
print(len(codigos))

81
81


In [6]:
df = pd.DataFrame(list(zip(nombres, codigos)))
df.head()

,0,1
0,[Emtricitabine],[DB00879]
1,[Zidovudine],[DB00495]
2,[Darunavir],[DB01264]
3,[Nevirapine],[DB00238]
4,[Valganciclovir],[DB01610]


In [8]:
df.to_csv("output/antivirales.csv")

In [10]:
df.columns = ["nombre", "código"]

In [12]:
df['SMILES']=df.apply(lambda x: getSmilesByName(x.nombre), axis=1)

In [13]:
df.head()

,nombre,código,SMILES
0,[Emtricitabine],[DB00879],C1C(OC(S1)CO)N2C=C(C(=NC2=O)N)F
1,[Zidovudine],[DB00495],CC1=CN(C(=O)NC1=O)C2CC(C(O2)CO)N=[N+]=[N-]
2,[Darunavir],[DB01264],CC(C)CN(CC(C(CC1=CC=CC=C1)NC(=O)OC2COC3C2CCO3)...
3,[Nevirapine],[DB00238],CC1=C2C(=NC=C1)N(C3=C(C=CC=N3)C(=O)N2)C4CC4
4,[Valganciclovir],[DB01610],CC(C)C(C(=O)OCC(CO)OCN1C=NC2=C1N=C(NC2=O)N)N


In [14]:
df.to_csv("output/antiviralesconsmiles.csv")